In [ ]:
import cv2
import os
import shutil

In [ ]:
video_file = 'fathers_day.mp4'
cascade_file = 'haarcascade_frontalface_default.xml'
result_dir = 'Result_720_1280'

In [ ]:
if os.path.exists(result_dir):
    shutil.rmtree(result_dir)

os.mkdir(result_dir)
os.mkdir(os.path.join(result_dir, 'Cropped'))

face_cascade = cv2.CascadeClassifier(cascade_file)

In [ ]:
cap = cv2.VideoCapture(video_file)
while not cap.isOpened():
    cap = cv2.VideoCapture(video_file)
    cv2.waitKey(1000)
    print "Wait for the header"

pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
with open(os.path.join(result_dir, 'Detections.txt'), 'w') as ann:
    while True:
        flag, frame = cap.read()
        if flag:
            #convert to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray,
                                                  scaleFactor=1.1,
                                                  minNeighbors=5,
                                                  minSize=(80, 80),
                                                  flags=cv2.CASCADE_FIND_BIGGEST_OBJECT)
            for (x, y, w, h) in faces:
                # annotation format: frame_number bbox_h bbox_y bbox_w bbox_h
                ann.write('%d %d %d %d %d\n'%(pos_frame, x, y, w, h))
                cv2.imwrite(os.path.join(result_dir, 'Cropped', '%04d.png'%(pos_frame)),frame[y:y+h, x:x+w])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # The frame is ready and already captured
            cv2.imshow('video', frame)
            pos_frame = cap.get(cv2.cv.CV_CAP_PROP_POS_FRAMES)
            if len(faces) > 0:
                print str(pos_frame)+" frame. Detected face size = [%d, %d]"%(w,h)
            else:
                print str(pos_frame) + " frame."
        else:
            # The next frame is not ready, so we try to read it again
            cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
            print "frame is not ready"
            # It is better to wait for a while for the next frame to be ready
            break

        if cv2.waitKey(10) == 27:
            break
